In [119]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from urllib.parse import urlparse

In [39]:
# Choose the executable path to driver 
executable_path = {'executable_path': '/Users/praty/Documents/Rice/Mars/All-About-Mars/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [30]:
#browser = init_browser()

In [20]:
# Set up url in chromedriver
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [31]:
# Reading the html page
html = browser.html
soup = BeautifulSoup(html, "html.parser")

## NASA Mars News

### Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [36]:
# Assign News title and text to variables
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text

print(f"News Title : {news_title}")
print(f"News Paragraph : {news_p}")

News Title : NASA Invites Students to Name Mars 2020 Rover
News Paragraph : Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


## JPL Mars Space Images - Featured Image

## Visit Jet Propulsion laboratory. Find and save the image url for the current featured Mars Image.

#### JPL URL : https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

In [138]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

In [96]:
jpl = browser.html
soup = BeautifulSoup(html_image, "html.parser")
img_text = soup.find("article", class_="carousel_item")["style"][:]

In [110]:
print(img_text)

background-image: url('/spaceimages/images/wallpaper/PIA19323-1920x1200.jpg');


In [127]:
import re
parsed_url = urlparse(jpl_url)
image_param = re.findall(r"'([^']*)'", img_text)[0]
image_param

'/spaceimages/images/wallpaper/PIA19323-1920x1200.jpg'

In [134]:
base_url = parsed_url.scheme +'://'+ parsed_url.netloc
base_url

'https://www.jpl.nasa.gov'

In [137]:
featured_image_url = base_url + image_param
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19323-1920x1200.jpg


## Mars Weather

### Visit the Mars Weather twitter account, Scrape and Save the latest Mars weather tweet from the page. 
#### Mars Weather twitter : https://twitter.com/marswxreport?lang=en

In [170]:
tw_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(tw_url)

In [171]:
html = browser.html
soup = BeautifulSoup(html,"html.parser")

In [181]:
mars_weather = soup.find("p",class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")#"js-tweet-text-container")

In [182]:
mars_weather.text


'We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/\xa0…pic.twitter.com/fWruE2v151'

## Mars Facts

#### Visit the Mars Facts webpage and scrape the table containing facts about the planet including Diameter, Mass, etc.
#### Mars Facts : https://space-facts.com/mars/

In [183]:
fact_url = 'https://space-facts.com/mars/'
browser.visit(fact_url)

In [197]:
# Read html page table into Pandas dataframe
mars_facts = pd.read_html(fact_url)

In [205]:
# Set up Mars Profile dataframe
mars_facts_df = mars_facts[1]
mars_facts_df.columns = ["Description", "Values"]
mars_facts_df = mars_facts_df.set_index(["Description"])

In [210]:
# Save table as html 
mars_facts_df.to_html('mars_facts.html')
mars_facts_df

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Mars Hemispheres

#### Visit the USGS Astrogeology site to obtain high resolution images for each of Mars's hemispheres. Save image URL and the image in full resolution. Store image url in a dictionary

In [285]:
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)

# Empty list to store all hemisphere image URL's
hemisphere_img_url = []
hemispheres_base_url = 'https://astrogeology.usgs.gov'

In [286]:
# Open USGS Page and store HTMl
usgs_html = browser.html
soup = BeautifulSoup(usgs_html, 'html.parser')

In [287]:
# Find all Image Item blocks
hemisphere = soup.find_all('div', class_='item')

In [288]:
for hem in hemisphere:
    
    # Store the title of each Hemisphere Image
    title = hem.find('h3').text
    img_url = hem.find('a')['href']
    
    # open the URL for each image to get full size
    browser.visit(hemispheres_base_url + img_url)
    
    # Store HTMl for Each Image Page
    img_html = browser.html
    soup = BeautifulSoup(img_html, 'html.parser')
    
    # Store URL for full size image by finding image class
    img_full_url = soup.find('img', class_='wide-image')['src']
    
    # Append title and Image URL to dictionary
    hemisphere_img_url.append({"title" : title, "img_url" : hemispheres_base_url + img_url})
    

In [289]:
hemisphere_img_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]